<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>



<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Système de Décision </center>
    <center> TP 0 : Prise en Main d'un Solveur SAT </center>
<hr style=" border:none; height:3px;">
</h1>



## Le format DIMACS

La plupart des solveurs SAT utilisent comme format de représentation de CNF le format DIMACS. C'est par exemple le format d'entrée pour la compétition [SAT](https://www.satcompetition.org/) dont les benchmarks de l'édition 2018 sont accessibles [ici](http://sat2018.forsyte.tuwien.ac.at/benchmarks/).

Un fichier DIMACS est composé de caractères ASCII et se veut être très simple à *parser* (attention, les fins de ligne sont au format Unix). Il est composé de 3 principales parties :
- Une première partie de commentaires. Un ligne de commentaire commence par le caractère `c`.
- Une deuxième partie détaillant le contenu du fichier. Il est de la forme `p FORMAT #VARIABLES #CLAUSES`
- L'ensemble des clauses. Chacune de ces lignes de clauses est composée d'entiers (positifs ou négatifs) et *devrait* se terminer par `0`. Un nombre négatif représente un littéral négatifs, tandis qu'un littéral positif est représenté par un nombre positif.

### Exemple

Si l'on considère le vocabulaire `[A, B, C]` (dans cet ordre), la formule **(¬C ∨ A)  ∧ (B ∨ ¬A ∨ C)** est représentée par le fichier suivant :


```
c
c Exemple de fichier .cnf file.
c
p cnf 3 2
-3 1 0
2 -1 3 0

```

## Le solveur gophersat

Le solveur **gophersat** est un programme open-source écrit en [Go](https://golang.org/). Ses sources sont accessibles sur [https://github.com/crillab/gophersat](https://github.com/crillab/gophersat). Une version compilée est disponible sur le EDUNAO du cours. C'est un solveur SAT, MAX-SAT et pseudo-booléen. Il est également capable de faire du comptage de modèles.

## Exemple :Problème de Coloration

Écrire un programme python permettant de générer le fichier `.cnf` permettant de résoudre le [problème de coloration de graphe](https://fr.wikipedia.org/wiki/Coloration_de_graphe) à 3 couleurs suivant (schéma issu de *wikipedia*) :

<img src="graph_3-coloring.png" alt="drawing" width="100"/>





* **Input** : 
    - un graphe non orienté sous forme $(V,E)$ où $E$ est un ensemble de paires de sommets et $V$ l'ensemble des sommets, et 
    - un entier $k$

* **Output** : une coloration du graphe en $k$ couleurs (un dictionnaire)
---


In [ ]:
# Exemple

#Ensemble V
Vertices = ['A','B','C','D']

#Ensemble E
Edges = [('A','B'),('A','C'),('B','C'),('B','D')]

#k=3
Colors = [1,2,3]

### Modélisation SAT
* **Variables** $x_{i,j}$ signifiant "le noeud $i$ a la couleur $j$"

In [ ]:
#Définitions des variables

variables = [(i,j) for i in Vertices for j in Colors]

#Création des dictionnaires

v2i = {v : i+1 for i,v in enumerate(variables)} #numérotation qui commence à 1
i2v = {i : v for v,i in v2i.items()}

print("variables to Integers")
print(v2i)

print("Intergers to Variables")
print(i2v)

* Clauses :
    - chaque noeud a au moins une couleur : $\forall i\in V\  x_{i,1} \lor \dots \lor x_{i,k}$
    - chaque noeud a au plus une couleur : $\forall i\in V\ \forall (j_1,j_2) : 1 \le j_1 < j_2 \le k\ \neg x_{i,j_1} \lor \neg x_{i,j_2}$
    - les sommets reliés par une arête ont des couleurs différentes : $\forall (v_1,v_2)\in E \forall j\in [k]\ \neg x_{v_1,k}\lor \neg x_{v_2,k}$



In [ ]:
#Clause 1

clauses_atleastonecolor = [[v2i[i,j] for j in Colors] for i in Vertices]
print(clauses_atleastonecolor)

In [ ]:
#Clause2

from itertools import combinations
clauses_atmostonecolor = [[-v2i[i,j1],-v2i[i,j2]]
                          for i in Vertices for j1,j2 in combinations(Colors,2)]
print(clauses_atmostonecolor)

In [ ]:
#Clause3

clauses_differentcolors = [[-v2i[v1,k],-v2i[v2,k]] for k in Colors for v1,v2 in Edges]
print(clauses_differentcolors)

## **Important**

- Unzip le dossuer gophersat.zip
- mettre le 

In [ ]:
#Construction du DIMCS et Résolution

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce markbook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [ ]:
#Lancer la résolution

myClauses= clauses_atleastonecolor + clauses_atmostonecolor + clauses_differentcolors
myDimacs = clauses_to_dimacs(myClauses,len(variables))

write_dimacs_file(myDimacs,"workingfile.cnf")
res = exec_gophersat("workingfile.cnf")

#Résultat
print(res)